# 28기 AI/Bigdata 아카데미 자연어처리 과목의 실습을 위한 견본 코드입니다.
본 코드는 주어진 한국어 hate-speech 데이터셋을 text classification task로 학습 및 추론을 수행합니다.

본 프로젝트에 참여하신 여러분은 코드를 수정하여 성능을 향상시키시면 됩니다.

코드 수정의 예시로는
*   하이퍼 패러미터 조정
*   전처리 과정 추가 혹은 변경 (이를테면 기사 제목 또한 학습에 사용)
*   토큰화 방법 조정

들이 있고, 이에 국한되지 않습니다.

먼저 모델과 데이터셋을 로드하고 학습하는데 필요한 패키지를 설치합니다.

In [14]:
! pip install datasets transformers accelerate evaluate

설치후 패키지를 로드해주고, 재현성을 위해 시드를 고정해줍니다.

In [15]:
import wandb
wandb.init(mode="disabled")

In [16]:
import transformers
from datasets import load_dataset

transformers.set_seed(42)

## 데이터 로드

이제 데이터셋을 로드해줍니다. \
kaggle에서 본 코드를 실행하신 경우, 데이터셋이 자동으로 런타임 루트 경로의 `kaggle/input`디렉토리에 마운트됩니다.

밑의 코드로 마운트된 경로의 데이터를 읽어옵니다.

만일 기사 제목을 추가적으로 학습에 사용하고 싶으시다면, \
같은 경로의 `train.news_title.txt`, `dev.news_title.txt`, `test.news_title.txt`도 추가적으로 로드후, \
Huggingface Datasets 패키지의 [`concatenate_datasets`](https://huggingface.co/docs/datasets/process#concatenate) 메서드를 활용하여 각 스플릿에 추가해주시면 됩니다.

In [17]:
# TODO: colab 테스트 때문에 경로 바꿈. /content/input/korean-hate-speech-detection/ 경로를 나중에 /kaggle/input/korean-hate-speech-detection/ 으로 바꾸기
train_dataset = load_dataset("csv", data_files="train.hate.csv")['train']
dev_dataset = load_dataset("csv", data_files="dev.hate.csv")['train']
test_dataset = load_dataset("csv", data_files="test.hate.no_label.csv")['train']

Generating train split: 7896 examples [00:00, 161158.46 examples/s]
Generating train split: 471 examples [00:00, 42091.39 examples/s]
Generating train split: 974 examples [00:00, 65273.17 examples/s]


데이터셋의 구조를 확인해봅시다. 만일 추가한 것이 없을 경우, `train` 스플릿에서는 `comments`와 `label` column 만이 있을 것입니다.

In [18]:
train_dataset

Dataset({
    features: ['comments', 'label'],
    num_rows: 7896
})

이제 한국어 hate-speech 데이터셋의 3가지 split `train`, `dev`, `test`을 모두 로드했습니다.

각 split에 대해 간단히 설명하자면, \
`train` split은 모델의 학습에 이용되어야 하고, \
`dev` split은 `train` split으로 학습한 모델의 hyper-parameter 조정 등을 위해 사용되어야 하고, \
`test` split은 튜닝이 완료된 최종 모델의 결과를 추론하는데 사용하는 데이터셋입니다.

`test` split의 결과는 모델 튜닝 중 사용되서는 안되며, 따라서 **레이블도 제공되지 않습니다**.

In [19]:
train_dataset[0] # 첫번쨰 데이터 확인

{'comments': '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....',
 'label': 'hate'}

## 데이터 전처리

이제 데이터셋을 전처리해줍니다.

본 코드는 예시 코드이기때문에, 간단하게 토크나이징만 진행합니다.

본 예시에서는 [`beomi/KcELECTRA-base`](https://huggingface.co/beomi/KcELECTRA-base)를 사용합니다. (모델 레포에 유용한 정보가 많이 있습니다.)\
만일 다른 모델을 사용하고 싶으시다면, [여기](https://huggingface.co/models?pipeline_tag=text-classification&language=ko)에서 다른 모델들을 해당 태스크에 적합한 다른 모델들을 찾아볼 수 있습니다.

In [20]:
model_checkpoint = 'beomi/KcELECTRA-base'

해당 모델의 토크나이저를 로드해줍니다.

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

이제 전처리에 사용할 함수를 정의해줍니다. 각 데이터는 `examples`라는 패러미터로 전달됩니다.

예시 전처리 함수는 토크나이징 및 `label` 데이터를 숫자로 매핑하는 처리만을 다룹니다.

In [22]:
label2id = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

In [23]:
def preprocess_function(examples):
    result_dict = tokenizer(examples['comments'], truncation=True)
    # 추가하고 싶은 전처리 과정이 있으시다면, 여기에 추가해주시면 됩니다.
    # 만일 기사 제목 데이터를 추가하셨다면, 밑의 예시 코드를 활용하여 두 column이 합쳐진 토크나이징 결과를 얻을 수 있습니다.
    # result_dict = tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True)
    if 'label' in examples.keys():
      result_dict['label'] = [label2id[label_str] for label_str in examples['label']]
    return result_dict

In [24]:
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['comments','label'])
dev_dataset = dev_dataset.map(preprocess_function, batched=True, remove_columns=['comments','label'])
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map: 100%|███████████████████████████| 974/974 [00:00<00:00, 3506.00 examples/s]


전처리 후, 데이터셋의 구조가 어떻게 바뀌었는지 확인해봅시다.

In [25]:
train_dataset

Dataset({
    features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 7896
})

이제 `train` 스플릿 기준 `input_ids`, `attention_mask`, `token_type_ids`, `label` 네 가지 column이 생긴 것을 확인하실 수 있습니다.

## 모델 학습

이제 선택한 모델을 학습해봅시다.
이를 위해 우선 모델을 로드해봅시다.

In [26]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 3  # none, offensive, hate 세 가지 이므로
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


이제 학습에 사용할 하이퍼 패리미터를 정의합니다.
하이퍼 패러미터를 조정하실 분은 밑의 코드를 변경하시면 됩니다.

In [27]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 16
learning_rate = 2e-5
epochs = 5
weight_decay = 0.01

args = TrainingArguments(
    f"{model_name}-finetuned-lr{learning_rate}-epochs{epochs}-decay{weight_decay}-hate",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    report_to="none"
)

import os
os.environ["WANDB_DISABLED"] = "true"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


이후 해당 하이퍼 패러미터를 비롯한 정보들을 Huggingface Trainer에 넘겨줍니다.

In [28]:
import evaluate

accuracy_metric = evaluate.load("accuracy")

def compute_metrics(p):
    logits, labels = p
    predictions = logits.argmax(axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy


In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

/tmp/ipykernel_5211/3846473124.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


이제 학습을 진행합니다. 기본 하이퍼 패리미터에서 `NVIDIA T4` GPU 기준 약 7~8분 정도 소요됩니다.

In [30]:
trainer.train()

/home/piai/anaconda3/envs/imageprocessing/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.808116,0.630573
2,No log,0.720663,0.685775
3,0.761000,0.779006,0.658174
4,0.761000,0.950549,0.641189
5,0.460900,0.900491,0.660297


/home/piai/anaconda3/envs/imageprocessing/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/piai/anaconda3/envs/imageprocessing/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/piai/anaconda3/envs/imageprocessing/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/piai/anaconda3/envs/imageprocessing/lib/python3.8/site-packages/torch/nn/parallel/

TrainOutput(global_step=1235, training_loss=0.5582856645468276, metrics={'train_runtime': 249.0352, 'train_samples_per_second': 158.532, 'train_steps_per_second': 4.959, 'total_flos': 993088706037120.0, 'train_loss': 0.5582856645468276, 'epoch': 5.0})

학습이 완료되었다면, Trainer의 `evaluate` 메서드를 이용해 `dev` 스플릿에서의 최종 loss를 확인합니다.

In [31]:
trainer.evaluate()

/home/piai/anaconda3/envs/imageprocessing/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.9004914164543152,
 'eval_accuracy': 0.6602972399150743,
 'eval_runtime': 1.1866,
 'eval_samples_per_second': 396.928,
 'eval_steps_per_second': 12.641,
 'epoch': 5.0}

만족할 만한 결과가 나오셨나요? 해당 결과를 kaggle에 업로드 하실려면, `dev` 스플릿의 결과가 아닌 `test` 스플릿의 결과가 필요합니다.

이를 위해 `test` 스플릿에서 추론 결과를 얻고, `test_results`에 저장합니다.

In [32]:
from tqdm import tqdm
import torch

test_results = []

for example_idx in tqdm(range(0, len(test_dataset), batch_size)):
    features = test_dataset[example_idx:example_idx + batch_size if example_idx + batch_size < len(test_dataset) else len(test_dataset)]
    batch_size = len(features['input_ids'])

    # 데이터의 배치 처리가 가능하게끔 padding을 진행합니다.
    input_ids_batch = []
    attention_mask_batch = []
    max_input_ids_len = max([len(feature) for feature in features['input_ids']])
    for input_ids, attention_mask in zip(features['input_ids'], features['attention_mask']):
        if len(input_ids) < max_input_ids_len:
            input_ids += [tokenizer.pad_token_id] * (max_input_ids_len - len(input_ids))
            attention_mask += [0] * (max_input_ids_len - len(attention_mask))
        input_ids_batch.append(input_ids)
        attention_mask_batch.append(attention_mask)
    features_batch = {'input_ids': torch.tensor(input_ids_batch).to('cuda'),
                      'attention_mask': torch.tensor(attention_mask_batch).to('cuda')}

    with torch.no_grad():
        model_output = model(**features_batch)
        preds = model_output.logits.argmax(dim=-1)
        preds = preds.squeeze().tolist()

    for comment, pred in zip(features['comments'], preds):
        test_results.append({'comments': comment, 'label': int(pred)})

100%|███████████████████████████████████████████| 61/61 [00:01<00:00, 46.77it/s]


모두 추론했다면, `test_result`에 저장된 결과를 csv 파일로 저장합니다.

In [33]:
import csv

with open('./submission.csv', 'w', newline='') as f:
    fieldnames = ['comments', 'label']
    w = csv.DictWriter(f, fieldnames=fieldnames)
    w.writeheader()
    w.writerows(test_results)

파일이 정확한 포맷으로 저장됬는지 확인해봅시다.


```
comments,label
(댓글 내용 1),(추론 결과 숫자)
(댓글 내용 2),(추론 결과 숫자)
...
```
의 형태로 저장되었으면 올바르게 저장된 것입니다.



In [34]:
!head -n 10 submission.csv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


comments,label
ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,1
둘다 넘 좋다~행복하세요,0
근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
장현승 얘도 참 이젠 짠하다...,1
신선하게 웃긴다ㅋㅋㅋ역시 동엽신~~!! 장소연님은 진짜 조선족인가 착각할정도로 말투가 리얼하네요,0
누군데 얘네?,1
"하자 인생들 모아다가 방송에 내보내고, 덜 하자가 교정해서 장사 풀리게 해주는 감동 스토리 백하자의 골목식당. 호텔 말고 그냥 하자 거리를 하나 열어서 거기다 하자 인생들 교화소를 만들지... 왜 저러고 살까...",2
진짜 라디오 스타 노래한거 보세요 홍진영은비비지도 못함,0
